# Auto-connect to google sheets documents

Based on

1. https://towardsdatascience.com/how-to-access-google-sheet-data-using-the-python-api-and-convert-to-pandas-dataframe-5ec020564f0e 
2. https://developers.google.com/sheets/api/quickstart/python.

### Our Data Sheet

https://docs.google.com/spreadsheets/d/1T6V1F8HP01w-yVQ-Ji95bIFnIeWvqfvS8YDxqozp6CI/edit#gid=0

In [125]:
from __future__ import print_function

import os
import pickle
import os.path
import numpy as np
import pandas as pd
from pathlib import Path

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

#### Function

In [126]:
def get_google_sheet(spreadsheet_id, range_name, credentials):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    return service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()


def gsheet2df(gsheet, header=0, stop=None):
    """ Converts Google sheet data to a Pandas DataFrame.
    Note: This script assumes that your data contains a header file on the first row!
    Also note that the Google API returns 'none' from empty cells - in order for the code
    below to work, you'll need to make sure your sheet doesn't contain empty cells,
    or update the code to account for such instances.
    """
    data_idx = header + 1
    header = gsheet.get('values', [])[header]   # Assumes first line is header!
    values = gsheet.get('values', [])[data_idx:stop]  # Everything else is data.
    n_vals = len(values)

    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []

            for row in values:
                try:
                    s = row[col_id]
                except IndexError as msg:
                    s = np.nan
                column_data.append(s)
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        df = pd.concat(all_data, axis=1)
        return df

#### Define paths

In [127]:
abspath = os.path.abspath('')
project_dir = str(Path(abspath).parents[0])

data_raw = os.path.join(project_dir, "data", "raw")
data_processed = os.path.join(project_dir, "data", "processed")

#### Authentification

In [128]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# <Your spreadsheet ID>
SPREADSHEET_ID = "1CVdDndGD1S8ab3D1KlXYcBcnjK8sBPl68Eth0KMgaEQ"

# <Your worksheet name>
RANGE_NAME = "Farmer 1"

# <Your worksheet names>
RANGE_NAMES = ["Farmer 1", "Farmer 2", "Forester 1", "Forester 1"]

# API credentials
credentials_fpath = os.path.join(project_dir, 'google_api_credentials.json')

#### Download

In [129]:
sheet_dict = {}

for i, sheet_name in enumerate(RANGE_NAMES):
    print(sheet_name)

    # 1) fetch data
    data_dict = get_google_sheet(
        credentials=credentials_fpath,
        spreadsheet_id=SPREADSHEET_ID,
        range_name=sheet_name
    )

    # 2) convert to data frame
    df_raw = gsheet2df(data_dict, header=0, stop=11)
    df_raw = df_raw.set_index("Round")

    # 3) convert to numeric
    df = df_raw.copy()
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="ignore")

    # 4) append to dict
    sheet_dict[sheet_name] = df

Farmer 1
Farmer 2
Forester 1
Forester 1


In [130]:
df_all = pd.concat(sheet_dict.values(), keys=sheet_dict.keys())
df_all.index = df_all.index.set_names(["player", "round"])
df_all = df_all.reset_index()

df_sheets = df_all.copy()
for col in df_sheets.columns:
    df_sheets[col] = pd.to_numeric(df_sheets[col], errors="ignore")

In [131]:
df_final = df_sheets.set_index(["round", "player"]).sort_index()
df_final

Plot  Native Forest  Farmland  Actions:   native forest  \
round player                                                               
1     Farmer 1     A1           50.0     -50.0        NaN            NaN   
      Farmer 2    NaN            NaN       NaN        NaN            NaN   
      Forester 1  NaN            NaN       NaN        NaN            NaN   
2     Farmer 1    NaN            NaN       NaN        NaN            NaN   
      Farmer 2    NaN            NaN       NaN        NaN            NaN   
      Forester 1  NaN            NaN       NaN        NaN            NaN   
3     Farmer 1    NaN            NaN       NaN        NaN            NaN   
      Farmer 2    NaN            NaN       NaN        NaN            NaN   
      Forester 1  NaN            NaN       NaN        NaN            NaN   
4     Farmer 1    NaN            NaN       NaN        NaN            NaN   
      Farmer 2    NaN            NaN       NaN        NaN            NaN   
      Forester 1  NaN            NaN       NaN        NaN            NaN   
5     Farmer 1    NaN            NaN       NaN        NaN            NaN   
      Farmer 2    NaN            NaN       NaN        NaN            NaN   
      Forester 1  NaN            NaN       NaN        NaN            NaN   
6     Farmer 1    NaN            NaN       NaN        NaN            NaN   
      Farmer 2    NaN            NaN       NaN        NaN            NaN   
      Forester 1  NaN            NaN       NaN        NaN            NaN   
7     Farmer 1    NaN            NaN       NaN        NaN            NaN   
      Farmer 2    NaN            NaN       NaN        NaN            NaN   
      Forester 1  NaN            NaN       NaN        NaN            NaN   
8     Farmer 1    NaN            NaN       NaN        NaN            NaN   
      Farmer 2    NaN            NaN       NaN        NaN            NaN   
      Forester 1  NaN            NaN       NaN        NaN            NaN   
9     Farmer 1    NaN            NaN       NaN        NaN            NaN   
      Farmer 2    NaN            NaN       NaN        NaN            NaN   
      Forester 1  NaN            NaN       NaN        NaN            NaN   
10    Farmer 1    NaN            NaN       NaN        NaN            NaN   
      Farmer 2    NaN            NaN       NaN        NaN            NaN   
      Forester 1  NaN            NaN       NaN        NaN            NaN   

                  farmland  keep everything  commercial forest  
round player                                                    
1     Farmer 1         NaN              NaN                NaN  
      Farmer 2         NaN              NaN                NaN  
      Forester 1       NaN              NaN                NaN  
2     Farmer 1         NaN              NaN                NaN  
      Farmer 2         NaN              NaN                NaN  
      Forester 1       NaN              NaN                NaN  
3     Farmer 1         NaN              NaN                NaN  
      Farmer 2         NaN              NaN                NaN  
      Forester 1       NaN              NaN                NaN  
4     Farmer 1         NaN              NaN                NaN  
      Farmer 2         NaN              NaN                NaN  
      Forester 1       NaN              NaN                NaN  
5     Farmer 1         NaN              NaN                NaN  
      Farmer 2         NaN              NaN                NaN  
      Forester 1       NaN              NaN                NaN  
6     Farmer 1         NaN              NaN                NaN  
      Farmer 2         NaN              NaN                NaN  
      Forester 1       NaN              NaN                NaN  
7     Farmer 1         NaN              NaN                NaN  
      Farmer 2         NaN              NaN                NaN  
      Forester 1       NaN              NaN                NaN  
8     Farmer 1         NaN              NaN                NaN  
      Farmer 2         N